In [49]:
# ======================================
# PDF → QUESTIONS → ANSWERS (RAG SYSTEM)
# ======================================

from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_ollama import OllamaEmbeddings, OllamaLLM
from langchain_core.prompts import PromptTemplate

# ================================
# 1. LOAD PDF
# ================================
loader = PyPDFLoader("data/dummy1.pdf")
docs = loader.load()

# LIMIT pages for speed (increase later)
docs = docs[:6]
print(f"Loaded pages: {len(docs)}")

# ================================
# 2. SPLIT INTO CHUNKS
# ================================
splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=150
)
chunks = splitter.split_documents(docs)

# LIMIT chunks for learning
chunks = chunks[:20]
print(f"Total chunks used: {len(chunks)}")

# ================================
# 3. EMBEDDINGS + VECTOR STORE
# ================================
embeddings = OllamaEmbeddings(model="nomic-embed-text")
vector_store = FAISS.from_documents(chunks, embeddings)
retriever = vector_store.as_retriever(search_kwargs={"k": 6})

print("Vector store ready ✅")

# ================================
# 4. LOAD LLM
# ================================
llm = OllamaLLM(
    model="gemma3:4b",
    temperature=0.2
)

# ================================
# 5. GET CONTEXT FROM PDF
# ================================
docs_for_context = retriever.invoke("Give important study content from this pdf")
context = "\n".join([d.page_content for d in docs_for_context])

# ================================
# 6. GENERATE QUESTIONS
# ================================
question_prompt = PromptTemplate(
    input_variables=["context"],
    template="""
You are an experienced teacher.

From the text below, create 5 important exam-oriented questions.

TEXT:
{context}

QUESTIONS:
"""
)

questions_text = llm.invoke(question_prompt.format(context=context))

print("\n================ QUESTIONS ================\n")
print(questions_text)

questions = [
    q.strip() for q in questions_text.split("\n")
    if q.strip() and q.strip()[0].isdigit()
]

# ================================
# 7. GENERATE ANSWERS (FAST & ACCURATE)
# ================================
answer_prompt = PromptTemplate(
    input_variables=["context", "questions"],
    template="""
Answer each question clearly using ONLY the given text.
If the answer is not in the text, say: "Not found in document".

TEXT:
{context}

QUESTIONS:
{questions}

Give numbered answers.
"""
)

answers_text = llm.invoke(
    answer_prompt.format(
        context=context[:4000],   # safety limit
        questions="\n".join(questions)
    )
)

print("\n================ ANSWERS ================\n")
print(answers_text)

print("\n🎉 DONE! PDF → Questions → Answers generated successfully")


Loaded pages: 6
Total chunks used: 6
Vector store ready ✅

================ QUESTIONS ================

Okay class, let’s solidify our understanding of this foundational unit. Based on the text provided, here are five exam-style questions designed to test your knowledge.  I’ve aimed for questions that require you to apply the concepts, not just recall definitions.

**Here are the questions:**

**1. (Conceptual Understanding - 10 Marks)**

   The text outlines two primary categories of data representation: numeric and non-numeric. Explain, in your own words, why it’s important for a computer system to be able to represent both types of data. Provide a specific example of a situation where representing non-numeric data is crucial.


**2. (Conversion & Application - 15 Marks)**

   Consider the following calculation: `X = 5, Y = 10, Z = X + Y`.  The text mentions Binary, Decimal, Hexadecimal, and Octal representations of numeric data.  Explain *why* these different number systems are used

In [48]:
# ================================
# 1. IMPORTS
# ================================
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_ollama import OllamaEmbeddings, OllamaLLM
from langchain_core.prompts import PromptTemplate

# ================================
# 2. LOAD PDF
# ================================
loader = PyPDFLoader("data/dummy1.pdf")
docs = loader.load()

# limit pages for learning
docs = docs[:2]
print(f"Loaded pages: {len(docs)}")

# ================================
# 3. SPLIT INTO CHUNKS
# ================================
splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100
)
chunks = splitter.split_documents(docs)

# limit chunks for fast testing
chunks = chunks[:5]
print(f"Total chunks used: {len(chunks)}")

# ================================
# 4. CREATE EMBEDDINGS + VECTOR STORE
# ================================
embeddings = OllamaEmbeddings(model="nomic-embed-text")
vector_store = FAISS.from_documents(chunks, embeddings)
retriever = vector_store.as_retriever()

print("Vector store ready ✅")

# ================================
# 5. LOAD LLM
# ================================
llm = OllamaLLM(
    model="gemma3:4b",
    temperature=0.2
)

# ================================
# 6. GENERATE QUESTIONS
# ================================
docs_for_questions = retriever.invoke("Generate questions from this pdf")
context = "\n".join([d.page_content for d in docs_for_questions])

question_prompt = PromptTemplate(
    input_variables=["context"],
    template="""
You are a teacher.

Based on the text below, generate 5 important questions.

TEXT:
{context}

QUESTIONS:
"""
)

questions_text = llm.invoke(question_prompt.format(context=context))
print("\n=== QUESTIONS ===\n")
print(questions_text)

# convert to list
questions = [q.strip() for q in questions_text.split("\n") if q.strip()]

# ================================
# 7. GENERATE ANSWERS (FAST VERSION)
# ================================
answer_prompt = PromptTemplate(
    input_variables=["context", "questions"],
    template="""
You are a teacher.

Answer the following questions ONLY using the given text.

TEXT:
{context}

QUESTIONS:
{questions}

Give clear numbered answers.
"""
)

all_questions = "\n".join(questions)

answer_text = llm.invoke(
    answer_prompt.format(
        context=context[:4000],  # limit context
        questions=all_questions
    )
)

print("\n=== ANSWERS ===\n")
print(answer_text)

print("\n🎉 DONE! PDF → Questions → Answers working successfully")


Loaded pages: 2
Total chunks used: 2
Vector store ready ✅

=== QUESTIONS ===

Okay class, let’s check our understanding of this introductory material. Based on this text, here are five important questions I want you to think about:

1.  **What is the primary focus of computer architecture, according to the text?** (This question targets the core definition provided.)

2.  **What does the text suggest is involved in the "interaction" of the functional modules within a computer?** (This probes for a deeper understanding of how the components work together.)

3.  **What are the key topics covered in Unit 1 of this course?** (This asks students to identify the specific areas of study within the unit.)

4.  **Why is understanding computer architecture important for meeting the “processing needs of the user”?** (This encourages critical thinking about the purpose of computer architecture.)

5.  **Can you briefly explain what “register transfer” and “micro-operations” might be related to with

In [46]:
docs = retriever.invoke("summary of this pdf")
context = "\n".join(d.page_content for d in docs)
context = context[:4000]

all_questions = "\n".join(
    [f"{i+1}. {q}" for i, q in enumerate(questions)]
)

prompt = f"""
You are a teacher.

Answer the following questions ONLY from the given text.

TEXT:
{context}

QUESTIONS:
{all_questions}

Give numbered answers clearly.
"""

response = llm.invoke(prompt)
print(response)


Okay, here are the numbered answers based on the provided text:

1.  Doyou
2.  What
3.  How
4.  When
5.  Where
6.  Why
7.  Who
8.  What
9.  When
10. Where
11. Why
12. Who
13. What
14. When
15. Where
16. Why
17. Who
18. What
19. When
20. Where
21. Why
22. Who
23. What
24. When
25. Where
26. Why
27. Who
28. What
29. When
30. Where
31. Why
32. Who
33. What
34. When
35. Where
36. Why
37. Who
38. What
39. When
40. Where
41. Why
42. Who
43. What
44. When
45. Where
46. Why
47. Who
48. What
49. When
50. Where
51. Why
52. Who
53. What
54. When
55. Where
56. Why
57. Who
58. What
59. When
60. Where
61. Why
62. Who
63. What
64. When
65. Where
66. Why
67. Who
68. What
69. When
70. Where
71. Why
72. Who
73. What
74. When
75. Where
76. Why
77. Who
78. What
79. When
80. Where
81. Why
82. Who
83. What
84. When
85. Where
86. Why
87. Who
88. What
89. When
90. Where
91. Why
92. Who
93. What
94. When
95. Where
96. Why
97. Who
98. What
99. When
100. Where



In [44]:
for q in questions:
    docs = retriever.invoke(q)
    context = "\n".join([d.page_content for d in docs])

    prompt = answer_prompt.format(
        context=context,
        question=q
    )

    answer = llm.invoke(prompt)
    answers.append({
        "question": q,
        "answer": answer
    })


KeyboardInterrupt: 

In [43]:
answer_prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="""
You are a teacher.

Answer the question ONLY using the context below.

Context:
{context}

Question:
{question}

Answer:
"""
)

In [42]:
answers = []

for q in questions:
    docs = retriever.invoke(q)
    context = "\n".join([d.page_content for d in docs])


In [39]:
prompt_text = question_prompt.format(context=context)
response = llm.invoke(prompt_text)
print(response)

Okay class, let’s check our understanding of these concepts. Based on the text, here are five important questions I want you to think about:

1.  **What is the fundamental process involved in converting a hexadecimal number like (10110.001)₂ to its binary equivalent, and why is it the reverse of binary to hexadecimal conversion?** (This question focuses on the core principle and the direction of the conversion.)

2.  **The text provides an example of converting (152A.25)₁₆ to binary. Can you explain, step-by-step, how the binary code (0001 0101 0010 1010.0010 0101)₂ was derived?** (This requires students to apply the conversion process to a specific example.)

3.  **What is a register in the context of computer architecture, and how does it relate to storing binary information?** (This question tests understanding of a key technical term and its function.)

4.  **The text mentions converting hexadecimal to decimal.  Briefly explain the method used to calculate the decimal value of a he

In [38]:
from langchain_core.prompts import PromptTemplate

question_prompt = PromptTemplate(
    input_variables=["context"],
    template="""
You are a teacher.

Based on the following text, generate 5 important questions.

TEXT:
{context}

QUESTIONS:
"""
)

In [37]:
context = "\n".join([d.page_content for d in docs])

In [36]:
retriever = vector_store.as_retriever()
docs = retriever.invoke("generate questions from this pdf")
context = "\n".join([d.page_content for d in docs])

In [33]:
from langchain_core.prompts import PromptTemplate

question_prompt = PromptTemplate(
    input_variables=["context"],
    template="""
You are a teacher.

Based on the following text, generate 5 important questions.

TEXT:
{context}

QUESTIONS:
"""
)

In [32]:
from langchain_ollama import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

# create embeddings
embeddings = OllamaEmbeddings(model="nomic-embed-text")

# create vector store
vector_store = FAISS.from_documents(chunks, embeddings)

In [31]:
final_prompt = prompt.format(text=text)
questions = llm.invoke(final_prompt)

print(questions)

Okay class, let’s check our understanding of this introductory material. Here are five questions based on the content we just covered – “Computer Data Representation & Register Transfer and Micro-operations” within the Diploma Studies in Computer Engineering unit:

1.  **What is the primary focus of Computer Architecture, according to the text?** (This question directly addresses the core definition provided and tests basic comprehension.)

2.  **Let’s consider a simple calculation: X=5, Y=10, Z=X+Y.  Explain, in your own words, what the variables X, Y, and Z represent in this scenario.** (This question requires students to apply the concept of variables and understand the relationship between the calculation and the data.)

3.  **The text mentions “Register Transfer and Micro-operations.” Can anyone briefly explain what you think these terms might relate to within a computer’s processing?** (This question encourages students to think about the lower-level operations happening inside a

In [30]:
from langchain_ollama import OllamaLLM

llm = OllamaLLM(
    model="gemma3:4b",
    temperature=0.3
)


In [29]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate(
    input_variables=["text"],
    template="""
You are a teacher.

Based on the following content, create 5 important questions.

CONTENT:
{text}

QUESTIONS:
"""
)


In [28]:
sample_chunks = chunks[:3]   # take first 3 chunks
text = "\n\n".join([c.page_content for c in sample_chunks])

In [27]:
type(chunks[0])


langchain_core.documents.base.Document

In [26]:
from langchain_text_splitters import TokenTextSplitter

splitter = TokenTextSplitter(
    chunk_size=200,      # tokens per chunk
    chunk_overlap=50     # overlap tokens
)

chunks = splitter.split_documents(docs)
print(f"Number of chunks: {len(chunks)}")

Number of chunks: 92


In [25]:
question_gen

'Diploma Studies\nComputer Engineering\nProf. Smit Thacker\nUnit – 1\nTitle - Computer Data \nRepresentation & Register \nTransfer and Micro-\noperations\nComputer Organization \n(09CE2401)\nComputer Data \nRepresentation, \nRegister Transfer and \nMicro-operations\n\uf076 Computer Architecture\n• Computer architecture is concerned with the structure and behavior of\nthe various functional modules of the computer and how they interact\nto provide the processing needs of the user.\n\uf076 Computer Architecture\nCalculator:\nX=5, y=10\nZ=x+y\nZ=5+10\nX, Y Z\nComputer Organization\n• Computer organization is concerned with the way the hardware\ncomponents are connected together to form a computer system.\nData Representation\n(1) Numeric data\n--Numbers (real, Integer)\n(2) Non-numeric data\n--letters, symbols\n(3) Data structures\n--list, tree, stack etc...\nProgram and many more...\nNumeric Data Representation\nBinary   Decimal Hexadecimal Octal\nDecimal\nDecimal digits (0, 1, 2, 3, 4, 

In [24]:
question_gen = ""
for page in data:
    question_gen += page.page_content + "\n"

In [23]:
len(data)

88

In [22]:
data

[Document(metadata={'producer': 'Microsoft® PowerPoint® 2016', 'creator': 'Microsoft® PowerPoint® 2016', 'creationdate': '2023-01-12T10:19:39+05:30', 'title': 'PowerPoint Presentation', 'author': 'ASUS', 'moddate': '2023-01-12T10:19:39+05:30', 'source': 'data/dummy1.pdf', 'total_pages': 88, 'page': 0, 'page_label': '1'}, page_content='Diploma Studies\nComputer Engineering\nProf. Smit Thacker\nUnit – 1\nTitle - Computer Data \nRepresentation & Register \nTransfer and Micro-\noperations\nComputer Organization \n(09CE2401)\nComputer Data \nRepresentation, \nRegister Transfer and \nMicro-operations'),
 Document(metadata={'producer': 'Microsoft® PowerPoint® 2016', 'creator': 'Microsoft® PowerPoint® 2016', 'creationdate': '2023-01-12T10:19:39+05:30', 'title': 'PowerPoint Presentation', 'author': 'ASUS', 'moddate': '2023-01-12T10:19:39+05:30', 'source': 'data/dummy1.pdf', 'total_pages': 88, 'page': 1, 'page_label': '2'}, page_content='\uf076 Computer Architecture\n• Computer architecture is c

In [21]:
file_path = "data/dummy1.pdf"
loader = PyPDFLoader(file_path)
data = loader.load()

In [20]:
%pwd

'c:\\Users\\Meet\\Desktop\\Gen-AI\\New folder\\LANGCHAIN-PROJECT'

In [19]:
from langchain_community.document_loaders import PyPDFLoader

In [18]:
from langchain_ollama import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

embeddings = OllamaEmbeddings(model="nomic-embed-text")
vector_store = FAISS.from_documents(chunks, embeddings)


In [17]:
from langchain_ollama import OllamaLLM
from langchain_core.prompts import PromptTemplate

# 1. Setup model
llm = OllamaLLM(
    model="gemma3:4b",
    temperature=0.3
)

# 2. Create template
template = """
You are an AI assistant.
Based on the following text, generate 3 important questions.

TEXT:
{content}

QUESTIONS:
"""

prompt = PromptTemplate(
    input_variables=["content"],
    template=template
)

# 3. Format + ask model
fake_text = "Artificial intelligence is shaping the future of technology..."
formatted_prompt = prompt.format(content=fake_text)

response = llm.invoke(formatted_prompt)
print(response)


Okay, based on the extremely brief text "Artificial intelligence is shaping the future of technology...", here are 3 important questions we could ask:

1.  **What specific areas of technology are being most significantly impacted by artificial intelligence?** (This probes for details beyond just a general statement.)
2.  **What are the potential benefits and risks associated with AI’s influence on technology?** (This explores the broader implications – positive and negative.)
3.  **How is the development and implementation of AI in technology evolving, and what trends are emerging?** (This asks about the current state and future direction of the field.)

Would you like me to generate questions based on a different text, or perhaps refine these based on a specific area of interest?


In [16]:
prompt_template = """You are a helpful AI assistant that helps people find information. You are given the following extracted parts of a long document and a question. Provide a conversational answer based on the context provided.
If you don't know the answer, just say you don't know. DO NOT try to make up an answer.
Context: {context}
Question: {question}
Helpful Answer:""" 

In [15]:
llm = OllamaLLM(model="gemma3:4b",
    temperature=0.3
    )

In [14]:
from langchain_ollama import OllamaLLM

In [13]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document

# 1. Load PDF
loader = PyPDFLoader("data/dummy1.pdf")
docs = loader.load()

# 2. Split into chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_documents(docs)

print(len(chunks))


98


In [12]:
text = "Hello Meet, you are learning AI and LangChain. This tool cuts a long text into smaller chunks."
chunks = splitter.split_text(text)
print(chunks)
print(type(chunks[0]))


['Hello Meet, you are learning AI and LangChain. This tool cuts a long text into smaller chunks.']
<class 'str'>


In [11]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100
)

print(splitter)

In [10]:
question_gen

'Diploma Studies\nComputer Engineering\nProf. Smit Thacker\nUnit – 1\nTitle - Computer Data \nRepresentation & Register \nTransfer and Micro-\noperations\nComputer Organization \n(09CE2401)\nComputer Data \nRepresentation, \nRegister Transfer and \nMicro-operations\uf076 Computer Architecture\n• Computer architecture is concerned with the structure and behavior of\nthe various functional modules of the computer and how they interact\nto provide the processing needs of the user.\uf076 Computer Architecture\nCalculator:\nX=5, y=10\nZ=x+y\nZ=5+10\nX, Y ZComputer Organization\n• Computer organization is concerned with the way the hardware\ncomponents are connected together to form a computer system.Data Representation\n(1) Numeric data\n--Numbers (real, Integer)\n(2) Non-numeric data\n--letters, symbols\n(3) Data structures\n--list, tree, stack etc...\nProgram and many more...Numeric Data Representation\nBinary   Decimal Hexadecimal OctalDecimal\nDecimal digits (0, 1, 2, 3, 4, 5, 6, 7, 8, 

In [9]:
question_gen = ""
for page in documents:
    question_gen += page.page_content

In [8]:
len(documents)

88

In [7]:
documents

[Document(metadata={'producer': 'Microsoft® PowerPoint® 2016', 'creator': 'Microsoft® PowerPoint® 2016', 'creationdate': '2023-01-12T10:19:39+05:30', 'title': 'PowerPoint Presentation', 'author': 'ASUS', 'moddate': '2023-01-12T10:19:39+05:30', 'source': 'data/dummy1.pdf', 'total_pages': 88, 'page': 0, 'page_label': '1'}, page_content='Diploma Studies\nComputer Engineering\nProf. Smit Thacker\nUnit – 1\nTitle - Computer Data \nRepresentation & Register \nTransfer and Micro-\noperations\nComputer Organization \n(09CE2401)\nComputer Data \nRepresentation, \nRegister Transfer and \nMicro-operations'),
 Document(metadata={'producer': 'Microsoft® PowerPoint® 2016', 'creator': 'Microsoft® PowerPoint® 2016', 'creationdate': '2023-01-12T10:19:39+05:30', 'title': 'PowerPoint Presentation', 'author': 'ASUS', 'moddate': '2023-01-12T10:19:39+05:30', 'source': 'data/dummy1.pdf', 'total_pages': 88, 'page': 1, 'page_label': '2'}, page_content='\uf076 Computer Architecture\n• Computer architecture is c

In [6]:
file_path = "data/dummy1.pdf"
loaders = PyPDFLoader(file_path)
documents = loaders.load()

In [5]:
%pwd

'c:\\Users\\Meet\\Desktop\\Gen-AI\\New folder\\LANGCHAIN-PROJECT'

In [4]:
%cd ..

c:\Users\Meet\Desktop\Gen-AI\New folder\LANGCHAIN-PROJECT


In [3]:
%pwd

'c:\\Users\\Meet\\Desktop\\Gen-AI\\New folder\\LANGCHAIN-PROJECT\\research'

In [2]:
from langchain_community.document_loaders import PyPDFLoader

In [1]:
from langchain_ollama import OllamaLLM
llm = OllamaLLM(model="gemma3:4b")

print(llm.invoke("Hello from local AI!"))


Hello back to you, Local AI! It’s great to hear from you. 😊 

What’s happening in your local area today? Is there anything interesting you’re noticing or any questions you have that I can help with? 

Let me know!
